In [ ]:
Get-ExecutionPolicy;
Get-ExecutionPolicy -List;
#Set-ExecutionPolicy AllSigned/RemoteSigned/Restricted/Unrestricted/Bypass

# AllSigned will require that all scripts run on the host be signed by a trusted publisher.  The script will simply not run in PowerShell 7. We get an error when trying to run a script
# previously created on the host if using PowerShell 5.

# RemoteSigned will allow scripts with trusted digital signatures from the internet and any scripts written on the local host to run. RemoteSigned is the default setting on Windows Servers.
# After downloading a script from the internet, attempting to run it with the execution policy RemoteSigned enabled will result in a prompt asking if you trust the script and want to run it.

# Restricted will prevent any scripts from running on the host. In PowerShell 7, the script will not run. In PowerShell 5, the script will not run, and we will see an error.

# Unrestricted will allow all scripts to run and will prompt the user if the script originated outside the local internet zone. This is the default setting and cannot be changed on
# non-Windows devices running PowerShell.
# The results are the same between PowerShell 5 and 7

# Bypass will allow all scripts to run and will not warn the user if the script is potentially malicious, use with caution.

In [ ]:
# Spelling and formatting errors can be common in larger script blocks, since Get-Process is specified as Get-Processes, the first Catch will fire when the code is executed
Try {
    Get-Processes
}
Catch [System.Management.Automation.CommandNotFoundException] {
    Write-Host "CATCH 1: The command is not found!" -ForegroundColor Red
}
Catch {
    Write-Host "CATCH 2: This is a catch all by default" -ForegroundColor Yellow
}
Finally {
    Write-Host "FINALLY: Runs no matter what..." -ForegroundColor Yellow
}

In [ ]:
# If we want to see the output without the error, we can use -ErrorAction SilentlyContinue
Try {
    $BiosName = (Get-CimInstance Win32_BIOS -ComputerName localhost,PRODUCE_ERROR -ErrorAction SilentlyContinue).Name
    Write-Host $BiosName -ForegroundColor Green
 }
 Catch [System.Management.Automation.CommandNotFoundException] {
     Write-Host "CATCH 1: The command is not found!" -ForegroundColor Red
 }
 Catch {
     Write-Host "CATCH 2: This is a catch all by default" -ForegroundColor Yellow
 }
 Finally {
     Write-Host "FINALLY: Runs no matter what..." -ForegroundColor Yellow
 }

In [ ]:
'''Trap Statements

    Trap Statement

        The Trap statement can also be used to handle terminating errors in scripts

        The Trap keyword specifies statements to run when a terminating error occurs

        Trap statements handle the terminating errors and allow execution to continue

'''
# Once again Get-Process is misspelled as Get-Processes and will cause an error at execution, the Trap Statement will trap the error and allow the remaining code to run successfully

Function Trap-Test {
    Trap [System.Management.Automation.CommandNotFoundException] {
        Write-Host "TRAP ERROR: The command is not found, check your spelling!" -ForegroundColor Red
    }
    Trap {
        Write-Host "TRAP ERROR: This is a catch all by default" -ForegroundColor Yellow
    }
    Get-Processes
    $BiosName = (Get-CimInstance Win32_BIOS -ComputerName localhost).Name
    Write-Host $BiosName -ForegroundColor Green
    }
    


In [ ]:
'''Error Reporting

    Another part of error handling in PowerShell is error reporting.

        Errors that occur during a PowerShell session are stored in the $Error and $LastExitCode variables and can be queried at any time to determine the cause of the errors.

        Particularly useful when running large scripts that may generate several errors that you do not get the chance to read as they scroll by in the window.

        Additionally, stores errors even if your ErrorAction preference is SilentlyContinue which we will discuss in the next section.

'''
# Using the $Error variable, we can see the error we just caused
$Error | Select-Object -First 1;

# Using the $LastExitCode variable. This will cause cmd.exe to open and if it is closed normally, it will return an exit code of 0. If any other number is returned, something failed.
$process = Start-Process -FilePath "C:\Windows\System32\cmd.exe" -PassThru
    $process.WaitForExit()

    $exitCode = $process.ExitCode

    Write-Host "Exit Code: $exitCode"

    if ($exitCode -eq 0) {
        Write-Host "Command executed successfully."
    } else {
        Write-Host "Error: Something went wrong with the process."
    }

'''

    Different ErrorActionPreference options and their functions

        Break - Enter the debugger when an error occurs or when an exception is raised.

        Continue: (Default) Displays the error message and continues executing.

        Ignore: Suppresses the error message and continues to execute the command. The Ignore value is intended for per-command use, not for use as saved preference. Ignore isnt a valid value for the $ErrorActionPreference variable.

        Inquire: Displays the error message and asks you whether you want to continue.

        SilentlyContinue: No effect. The error message isnt displayed, and execution continues without interruption.

        Stop: Displays the error message and stops executing. In addition to the error generated, the Stop value generates an ActionPreferenceStopException object to the error stream.

        Suspend: Automatically suspends a workflow job to allow for further investigation. After investigation, the workflow can be resumed. The Suspend value is intended for per-command use, not for use as saved preference. Suspend isnt a valid value for the $ErrorActionPreference variable.

'''


In [ ]:
# Script Name: ProcessesAndNetwork.ps1
# Description: This script uses several of the concepts learned in this course to display processes or network connections to the user.

function Show-ProcessesOrNetwork {

    # Parameters to store the process name or port number supplied by the user at run time
    param (
        [string]$ProcessName,
        [int]$PortNumber
    )

    $continue = $true

    # While loop to run the script until the user enters the choice to exit or presses Ctrl + C
    while ($continue) {
        Write-Host "Choose an option:"
        Write-Host "1. Show Running Processes"
        Write-Host "2. Show Network Connections"
        Write-Host "3. Show Processes by Name"
        Write-Host "4. Show Network Connections by Port"
        Write-Host "5. Exit"

        $choice = Read-Host "Enter 1, 2, 3, 4, or 5"

        # Switch construct that provides the user with different outcomes based on their choice
        switch ($choice) {
            1 {
                Write-Host "Running Processes:"
                Get-Process
                break
            }
            2 {
                Write-Host "Network Connections:"
                Get-NetTCPConnection
                break
            }
            3 { # Prompt the user for a Process Name to search for with the Get-Process cmdlet
                if (-not $ProcessName) {
                    $ProcessName = Read-Host "Enter the name of the process to search for"
                }
                Write-Host "Processes matching '$ProcessName':"
                Get-Process -Name $ProcessName

                # Resets the process name to an empty string so the user can enter a new one
                $ProcessName = ""
                break
            }
            4 { # Prompt the user for a port number to search for with the Get-NetTCPConnection cmdlet
                if (-not $PortNumber) {
                    $PortNumber = Read-Host "Enter the network port to search for"
                }
                Write-Host "Network Connections on port:"$PortNumber
                Get-NetTCPConnection | Where-Object { $_.LocalPort -eq $PortNumber -or $_.RemotePort -eq $PortNumber } | Format-List

                # Resets the port number to an empty string so the user can enter a new one
                $PortNumber = ""
                break
            }
            5 {
                Write-Host "Exiting..."
                $continue = $false
                break
            }
            default {
                Write-Host "Invalid choice. Please enter 1, 2, 3, 4, or 5. Try again."
            }
        }
    }
}

Show-ProcessesOrNetwork

In [ ]:
# Script Name: ProcessesAndNetworkRemote.ps1
<# Description: This script uses several of the concepts learned in this course to display processes or network connections to the user on the local host.
It is a repurposed version of ProcessesAndNetwork.ps1 script that includes remote functionality and some event handling using more concepts learned during this course. #>

function Show-ProcessesOrNetwork {

    # Gets domain credentials for PSRemoting
    $Creds = Get-Credential
    # Stores the current value of your trusted hosts file for later use.
    $WSMan = Get-Item WSMan:\localhost\Client\TrustedHosts | Select-Object Value
    # Allow your host to connect to multiple hosts using PSRemoting.
    Set-Item WSMan:\localhost\Client\TrustedHosts -Value * -Force

    # Stores user provided IP addresses into a variable, splits them on the comma, and gets each unique IP and stores them in a separate variable.
    $IPAddresses = Read-Host "Enter one or more IP addresses (separated by commas)"
    $IPAddresses = $IPAddresses -split ','
    $UniqueIPs = $IPAddresses | Get-Unique

    # Event handler in the case CTRL+C is pressed during run time using a Try/Finally construct.
    Try {

        # Ensures the script will continue to run after each selection is made until option 5 is chosen or Ctrl+C is sent.
        $continue = $true

        # While Loop that will continue as long as the $continue variable resolves to $true.
        while ($continue) {
            Write-Host "Choose an option:`n1. Show Running Processes`n2. Show Network Connections`n3. Show Processes by Name`n4. Show Established Network Connections by Port`n5. Exit"

            # Reads user input for their selection.
            $choice = Read-Host "Enter 1, 2, 3, 4, or 5"

            # Switch construct that provides the user with different outcomes based on their choice
            switch ($choice) {
                1 { # Foreach Loop that Checks a remote computer or computers for all running processes.
                    foreach ($IP in $UniqueIPs) {
                        $result = Invoke-Command -ComputerName $IP -ScriptBlock {Get-Process | Select-Object -Property ProcessName,ID,Path | Format-Table} -Credential $Creds -ErrorAction SilentlyContinue

                        if ($result) {
                            Write-Host "Showing Processes Running on ""$IP"":"`n
                            $result | ForEach-Object { $_ }
                        }
                    }
                    break
                }
                2 { # Foreach Loop that Checks a remote computer or computers for network connections.
                    foreach ($IP in $UniqueIPs) {
                        $result = Invoke-Command -ComputerName $IP -ScriptBlock {
                            Get-NetTCPConnection |
                            Select-Object -Property LocalAddress,RemoteAddress,State,LocalPort,RemotePort |
                            Where-Object {
                                $_.LocalAddress -ne "0.0.0.0" -and $_.LocalAddress -ne "::" -and $_.LocalAddress -notlike "fe*" -and
                                $_.RemoteAddress -ne "0.0.0.0" -and $_.RemoteAddress -notlike "::*" -and $_.RemoteAddress -notlike "fe*"
                            } |
                            Format-Table
                        } -Credential $Creds -ErrorAction SilentlyContinue

                        if ($result) {
                            Write-Host "Showing Network Connections on ""$IP"":" `n
                            $result | ForEach-Object { $_ }
                        }
                    }
                    break
                }
                3 { # If statement to check that the ProcessName parameter is empty and if it is, it asks the user to provide a process name.
                    if (-not $ProcessName) {
                        $ProcessName = Read-Host "Enter the name of the process to search for"
                    }

                    # Foreach Loop that Checks a remote computer or computers for running processes that match the user provided input.
                    foreach ($IP in $UniqueIPs) {
                        $result = Invoke-Command -ComputerName $IP -ScriptBlock {
                            param($ProcessName)
                            Get-Process -Name $ProcessName -ErrorAction SilentlyContinue | Select-Object -Property ProcessName, ID, Path
                        } -ArgumentList $ProcessName -Credential $Creds

                        if ($result) {
                            Write-Host "$ProcessName is Running on ""$IP""" `n
                            $result | Select-Object -Property PSComputerName,ProcessName, ID, Path | Format-Table
                        } else {
                            Write-Host "$ProcessName is not Running on ""$IP""" `n
                        }

                        $ProcessName = ""
                    }

                    break
                 }
                 4 { # If statement to check that the PortNumber parameter is empty and if it is, it asks the user to provide a port number.
                    if (-not $PortNumber) {
                        $PortNumber = Read-Host "Enter the network port to search for"
                    }

                    # Foreach Loop that Checks a remote computer or computers for established connections on a port provided by the user.
                    foreach ($IP in $UniqueIPs) {
                        $result =  Invoke-Command -ComputerName $IP -Credential $Creds -ScriptBlock {
                            param($PortNumber)
                            Get-NetTCPConnection | Where-Object { $_.LocalPort -eq $PortNumber -and $_.State -eq "established" -or $_.RemotePort -eq $PortNumber -and $_.State -eq "established"  }
                        } -ArgumentList $PortNumber

                        if ($result) {
                            Write-Host "Network Connections on port $PortNumber on ""$IP""" `n
                            $result | Format-List
                        } else {
                            Write-Host "Port $PortNumber not found in any connections on ""$IP""" `n
                        }

                        $PortNumber = ""

                    }

                    break
                }
                5 { # Exit option that will exit the script gracefully.
                    Write-Host "Exiting..."
                    $continue = $false
                    break
                }
                default { # Default option will trigger if any invalid input is provided when the user is selecting a choice.
                    Write-Host "Invalid choice. Please enter 1, 2, 3, 4, or 5. Try again.`n"
                }
            }


        }
    }
    Finally {
        # Return WSMan to its original setting even if a CTRL+C interrupt event takes place.
        Set-Item WSMan:\localhost\Client\TrustedHosts -Value $WSMan.Value -Force
    }
}

# Calls the primary function of the script.
Show-ProcessesOrNetwork

In [ ]:
#Question 11 Scripting 2
$creds = Get-Credential
invoke-command -computername 10.50.35.169 -credential $creds -scriptblock{get-itempropert 'HKLM:\Software\Microsoft\Windows\CurrentVersion\Run'}

In [ ]:
#Question 12 Scripting 3
'''Write a PowerShell script that will remotely query network information for 10.50.35.169 and return the results. 
The flag is the suspicious port number that is in a listening state.'''

invoke-command -computername 10.50.35.169 -credential $creds -scriptblock{get-nettcpconnection | where-object 'State' -eq 'Listen'}

In [ ]:
#Question 13 Script Repurposing 1
<#Using the PowerShell script you created to find the listening port, repurpose that script to determine the process ID of the process that is opening the suspicious port. 
Once you find the process ID, determine the name of the executable. 
The flag is the name of the executable.#>

invoke-command -computername 10.50.35.169 -credential $creds -scriptblock{get-nettcpconnection | where-object 'State' -eq 'Listen'}

In [ ]:
#Question15 Scripting 4
$result = Invoke-Command -ComputerName 10.50.35.169 -credential $creds -ScriptBlock {
    Get-content 'C:\Users\Defender\Documents\SuperSecretSauce.rtf'} 

$result

In [ ]:
#Question16 Scripting 5
Invoke-Command -ComputerName 10.50.35.169 -Credential $Creds -ScriptBlock {$Content = 
    "cmd.exe /c ping -n 1 10.50.35.169"; $FilePath = "C:\Users\Defender\Desktop\Cook_24-501.ps1"; Set-Content -Path $FilePath -Value $Content}

In [ ]:
#Question 17 Scripting 6
<#Using PowerShell, write code that will create a PowerShell script on 10.50.35.169. Name your script LastName_ClassNumber.ps1, ex. Doe_24-001.ps1.#>
Invoke-Command -ComputerName 10.50.35.169 -Credential $Creds -ScriptBlock {$Content = "cmd.exe /c ping -n 1 10.50.35.169"; $FilePath = "C:\Users\Defender\Desktop\Cook_24-501.ps1"; Set-Content -Path $FilePath -Value $Content}
Invoke-Command -ComputerName 127.0.0.1 -Credential $Creds -ScriptBlock {$Process = get-process | format-table; $network = get-nettcpconnection | format-table; $FilePath = "C:\Users\Student\Desktop\Cook_24-501.txt"; Set-Content -Path $FilePath -Value $Process; add-content -Path $FilePath -Value $network}

In [ ]:
Invoke-Command -ComputerName 10.50.35.169 -Credential $Creds -ScriptBlock {powershell.exe "C:\Users\Defender\Desktop\Cook_24-501.ps1" | 
    $FilePath = "C:\Users\Defender\Desktop\Cook_24-501.txt";
    Set-Content -Path $FilePath -Value $Process; 
    add-content -Path $FilePath -Value $network}

In [ ]:
Invoke-Command -ComputerName 127.0.0.1 -Credential $Creds -ScriptBlock {$Process = get-process | select-object name,id,path; 
    $network = get-nettcpconnection | select-object localport, remoteport, owningprocess;
    $FilePath = "C:\Users\Student\Desktop\Cook_24-501.txt";
    Set-Content -Path $FilePath -Value $Process; 
    add-content -Path $FilePath -Value $network;
}


In [ ]:
Invoke-Command -ComputerName 10.50.35.169 -Credential $Creds -ScriptBlock {
    $Content = "$Process = get-process | select-object name,id,path; $network = get-nettcpconnection | select-object localport, remoteport, owningprocess;" 
    $FilePath = "C:\Users\Defender\Desktop\Cook_24-501.ps1"; Set-Content -Path $FilePath -Value $Content;
    powershell.exe 'C:\Users\Defender\Desktop\Cook_24-501.ps1'

};
    

In [ ]:
invoke-command -computername 10.50.35.169 -credential $creds -scriptblock{get-process | select-object name,id,status,path | where-object name -like "*defender*"| format-table}

In [ ]:
PS C:\Users\student> Invoke-Command -ComputerName 10.50.35.169 -Credential $Creds -ScriptBlock {powershell.exe "C:\Users\Defender\Desktop\Cook_24-501.ps1";
>>     $FilePath = "C:\Users\Defender\Desktop\Cook_24-501.txt";
>>     Set-Content -Path $FilePath -Value $Process;
>>     add-content -Path $FilePath -Value $network}


In [ ]:
Invoke-Command -ComputerName 10.50.35.169 -Credential $Creds -ScriptBlock {
    $source = "c:\Users\Defender\Desktop\Cook_24-501.txt";
    $destination = "\\192.168.65.10\Users\Desktop";
    copy-item -path $source -destination $destination
}